# Этап 2: Загрузка статей

**Файл:** `data_parse.ipynb`

## Метаданные

Каждая статья содержит:
- `arxiv_id` — уникальный идентификатор
- `title` — название
- `authors` — список авторов
- `summary` — аннотация
- `categories` — категории arXiv
- `published` / `updated` — даты публикации
- `pdf_url` — ссылка на PDF

---

In [1]:
import pandas as pd
import arxiv
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import ssl
import certifi
import urllib.request

ctx = ssl.create_default_context(cafile=certifi.where())
https_handler = urllib.request.HTTPSHandler(context=ctx)
opener = urllib.request.build_opener(https_handler)
urllib.request.install_opener(opener)

In [2]:
METADATA_FILE = 'data/metadata/papers_metadata.csv'
TEX_DIR = Path('data/raw/tex')
NUM_WORKERS = 12

TEX_DIR.mkdir(parents=True, exist_ok=True)

client = arxiv.Client(
    page_size=100,
    delay_seconds=3,
    num_retries=3
)

In [3]:
print("📋 Загружаем метаданные...")
df_all = pd.read_csv(METADATA_FILE)

print("🔍 Проверяем скачанные файлы...")
existing_files = set()
for f in TEX_DIR.glob('*.gz'):
    parts = f.name.split('.')
    if len(parts) >= 2:
        arxiv_id = f"{parts[0]}.{parts[1]}"
        existing_files.add(arxiv_id)

df_to_download = df_all[~df_all['arxiv_id'].isin(existing_files)].copy()

print("\n" + "="*70)
print("📊 СТАТИСТИКА")
print("="*70)
print(f"📚 Всего в датасете:     {len(df_all):>6}")
print(f"✅ Уже скачано:          {len(existing_files):>6}")
print(f"📥 Нужно скачать:        {len(df_to_download):>6}")
print(f"📈 Прогресс:             {len(existing_files)/len(df_all)*100:>5.1f}%")
print("="*70)

📋 Загружаем метаданные...
🔍 Проверяем скачанные файлы...

📊 СТАТИСТИКА
📚 Всего в датасете:      16503
✅ Уже скачано:           16412
📥 Нужно скачать:            91
📈 Прогресс:              99.4%


In [4]:
def download_paper(arxiv_id):
    try:
        search = arxiv.Search(id_list=[arxiv_id])
        paper = next(client.results(search))
        paper.download_source(dirpath=str(TEX_DIR))
        return ('success', arxiv_id, None)
    except Exception as e:
        return ('error', arxiv_id, str(e))

In [5]:
if len(df_to_download) == 0:
    print("Все статьи уже скачаны!")
else:
    print(f"Запускаем скачивание {len(df_to_download)} статей...\n")
    
    success_count = 0
    error_count = 0
    errors_list = []
    
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = [
            executor.submit(download_paper, row['arxiv_id']) 
            for _, row in df_to_download.iterrows()
        ]
        
        for future in tqdm(futures, total=len(df_to_download), 
                          desc="Скачивание", unit="статья"):
            status, arxiv_id, error = future.result()
            
            if status == 'success':
                success_count += 1
            else:
                error_count += 1
                errors_list.append({'arxiv_id': arxiv_id, 'error': error})
                tqdm.write(f"❌ {arxiv_id}: {error}")
    
    print("\n🏁 Скачивание завершено!")

Запускаем скачивание 91 статей...



Скачивание:   7%|████▍                                                              | 6/91 [00:01<00:11,  7.68статья/s]

❌ 2501.18957v2: HTTP Error 404: Not Found
❌ 2501.18826v2: HTTP Error 404: Not Found
❌ 2501.18205v3: HTTP Error 404: Not Found
❌ 2501.17386v2: HTTP Error 404: Not Found
❌ 2501.16658v2: HTTP Error 404: Not Found
❌ 2501.17187v2: HTTP Error 404: Not Found
❌ 2501.15405v2: HTTP Error 404: Not Found
❌ 2501.14936v2: HTTP Error 404: Not Found
❌ 2501.14119v2: HTTP Error 404: Not Found
❌ 2501.13999v2: HTTP Error 404: Not Found


Скачивание:   7%|████▍                                                              | 6/91 [00:01<00:11,  7.68статья/s]

❌ 2501.12901v2: HTTP Error 404: Not Found
❌ 2501.11417v2: HTTP Error 404: Not Found


...

❌ 2508.02013v5: HTTP Error 404: Not Found
❌ 2508.00370v2: HTTP Error 404: Not Found
❌ 2509.24130v2: HTTP Error 404: Not Found
❌ 2509.23188v2: HTTP Error 404: Not Found
❌ 2509.15640v2: HTTP Error 404: Not Found
❌ 2509.08022v2: HTTP Error 404: Not Found
❌ 2509.03888v2: HTTP Error 404: Not Found


Скачивание:  86%|████████████████████████████████████████████████████████▌         | 78/91 [00:20<00:02,  4.36статья/s]

❌ 2510.23252v2: HTTP Error 404: Not Found
❌ 2510.15843v2: HTTP Error 404: Not Found
❌ 2510.15349v2: HTTP Error 404: Not Found
❌ 2510.09243v2: HTTP Error 404: Not Found
❌ 2510.08158v2: HTTP Error 404: Not Found


Скачивание: 100%|██████████████████████████████████████████████████████████████████| 91/91 [00:24<00:00,  3.72статья/s]

❌ 2510.07019v2: HTTP Error 404: Not Found
❌ 2510.04983v3: HTTP Error 404: Not Found
❌ 2510.02855v4: HTTP Error 404: Not Found
❌ 2510.01612v2: HTTP Error 404: Not Found
❌ 2511.08230v2: HTTP Error 404: Not Found
❌ 2511.04108v2: HTTP Error 404: Not Found
❌ 2512.10398v2: HTTP Error 404: Not Found

🏁 Скачивание завершено!


In [6]:
total_files = len(list(TEX_DIR.glob('*.gz')))

print("\n" + "="*70)
print("ИТОГОВАЯ СТАТИСТИКА")
print("="*70)
print(f"Успешно скачано:      {success_count:>6}")
print(f"Ошибок:               {error_count:>6}")
print(f"Всего .gz файлов:     {total_files:>6}")
print(f"Завершено:            {total_files/len(df_all)*100:>5.1f}%")
print("="*70)


ИТОГОВАЯ СТАТИСТИКА
Успешно скачано:           0
Ошибок:                   91
Всего .gz файлов:      16412
Завершено:             99.4%
